## Generated Dataset Analysis

As we obversaved that hallucination exists in LVM, the quality of the instruct dataset generated by LLM is needed to be checked.

We use a more powerful LLM Cluade2.1 to do quality evaluation.

In [2]:
import os
from langchain.llms.bedrock import Bedrock
from utils import get_bedrock_client

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.1,
    "stop_sequences": ["\n\nHuman"],
}

boto3_bedrock = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [3]:
def format_prompt_cluade(prompt):
    return f"""\n\nHuman: {prompt}\n\nAssistant:"""

In [4]:
import json

with open('llava_v1_5_mix665k.json', 'r') as f:
    mix665k = json.load(f)
    
with open('conv_mixtral.json', 'r') as f:
    conv_mixtral = json.load(f)
    
with open('caps_coco.json', 'r') as f:
    caps_coco = json.load(f)

In [5]:
def convert_dict(org_dict):
    new_dict = {}

    turn_num = []
    for ins_data in org_dict:
        if ins_data['id'] in new_dict:
            new_dict[ins_data['id']].append(ins_data)
        else:
            new_dict[ins_data['id']] = [ins_data]
            turn_num.append(len(ins_data['conversations'])/2)
    return new_dict

In [6]:
img_ids = list(caps_coco.keys())

In [7]:
mix665_dict = convert_dict(mix665k)
conv_gpt4 = {k:v[0] for k, v in mix665_dict.items() if k in img_ids}

In [8]:
conv_gpt4

{'000000033471': {'id': '000000033471',
  'image': 'coco/train2017/000000033471.jpg',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat are the colors of the bus in the image?'},
   {'from': 'gpt', 'value': 'The bus in the image is white and red.'},
   {'from': 'human',
    'value': 'What feature can be seen on the back of the bus?'},
   {'from': 'gpt', 'value': 'The back of the bus features an advertisement.'},
   {'from': 'human',
    'value': 'Is the bus driving down the street or pulled off to the side?'},
   {'from': 'gpt',
    'value': 'The bus is driving down the street, which is crowded with people and other vehicles.'}]},
 '000000052846': {'id': '000000052846',
  'image': 'coco/train2017/000000052846.jpg',
  'conversations': [{'from': 'human',
    'value': 'Where is the cat positioned in the image?\n<image>'},
   {'from': 'gpt',
    'value': 'The cat is positioned on top of the back of the couch in the living room.'},
   {'from': 'human', 'value': 'What is the 

In [9]:
def combine_qa(qa_dict):
    qa_dict_new = {}
    for k, v in qa_dict.items():
        if isinstance(v, list):
            v = v[0]
        qa_str = ""
        for conv in v['conversations']:
            if conv['from'] == 'human':
                qa_str += f"Question:{conv['value']}\n"
            else:
                qa_str += f"Anwser:{conv['value']}\n"
        qa_dict_new[k] = qa_str
    return qa_dict_new

In [10]:
conv_gpt4_dict = combine_qa(conv_gpt4)

In [11]:
conv_mixtral_dict = convert_dict(conv_mixtral)

In [12]:
conv_mixtral_dict_new = combine_qa(conv_mixtral_dict)

In [13]:
cap_qa_list = []
for k, v in caps_coco.items():
    cap_qa_list.append({
        'capion': ' '.join(v),
        'id': k,
        'mixtral': conv_mixtral_dict_new[k],
        'gpt4': conv_gpt4_dict[k]
    })


In [14]:
cap_qa_list

[{'capion': 'A white and red bus driving down a city street. A transit bus riding past some people in a parking area. A huge white bus with advertisement on the back is driving down the road. The city bus is pulled off to the side of the road.  a big bus is driving down a crowded street',
  'id': '000000033471',
  'mixtral': 'Question:<image>\nWhat type of vehicle is depicted in the image?\nAnwser:The image shows a city bus with white and red colors\nQuestion:What is the size of the bus?\nAnwser:The bus is quite large, as described in the third sentence "A huge white bus with advertisement on the back is driving down the road."\nQuestion:What color is the bus?\nAnwser:The bus is white and red, as stated in the first sentence\nQuestion:Where is the bus driving?\nAnwser:The bus is driving down a city street, as mentioned in the first sentence and seen in the image\nQuestion:Are there people around the bus in the picture?\nAnwser:Yes, in the second sentence, "A transit bus riding past som

In [15]:
with open('cap_qa_list.txt', 'w') as f:
    json.dump(cap_qa_list, f, indent=4)

### Assess the generated instruct data

In [16]:
prompt = f"""
I used GPT4 and Mixtral to generate QA conversation based on the caption of images. Please assess each result generated by both LLM.
1. Accuracy: This measures how well the answers reflect the information provided in the caption.
2. Relevance: This metric evaluates if the answers are pertinent to the questions asked.
3. Consistency: This assesses whether the answers are consistent with each other and with the caption.
4. Detail: This looks at how comprehensive the answers are.
5. Comprehensiveness: This metric evaluates the extent to which the QA conversation addresses and incorporates all the details provided in the caption.
   It assesses whether the responses acknowledge each descriptive element and whether any aspect of the caption is omitted or overlooked in the answers.
   
{cap_qa_list}
"""

format_prompt = format_prompt_cluade(prompt)

In [17]:
textgen_llm(format_prompt)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" Here is my assessment of the results generated by Mixtral and GPT4 for the image caption QA:\n\n1. Accuracy\n\nMixtral tends to provide more accurate and comprehensive answers that correctly reflect the information in the captions. Its responses cover more descriptive details and align closely with the content provided. \n\nGPT4 answers are generally accurate as well, but lack some specificity at times. There are a few cases where additional details could have been incorporated from the captions.\n\n2. Relevance  \n\nBoth Mixtral and GPT4 provide relevant responses to the questions asked. Their answers directly pertain to the query and image details.\n\n3. Consistency\n\nMixtral demonstrates stronger consistency across its responses. Its answers don't contain contradictions and align with the facts provided in the captions. \n\nGPT4 is mostly consistent, but there are 1-2 instances where an answer could be interpreted as slightly inconsistent with an earlier response.\n\n4. Detail\n\